In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
# Import the data we just downloaded and installed.
from nltk.corpus import movie_reviews, stopwords

In [4]:
print ('total reviews', len(movie_reviews.fileids()))
print ('categories', movie_reviews.categories())
print ('positive count', len(movie_reviews.fileids('pos')))
print ('negative count', len(movie_reviews.fileids('neg')))

total reviews 2000
categories ['neg', 'pos']
positive count 1000
negative count 1000


In [5]:
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid), category))
 
print(len(documents))
print('')
print(documents[0])

2000

(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg')


In [6]:
from random import shuffle 
shuffle(documents)

In [7]:
all_words = [word.lower() for word in movie_reviews.words()]

In [8]:
from nltk import FreqDist
 
all_words_frequency = FreqDist(all_words)
 
print (all_words_frequency)
print('')
# print 10 most frequently occurring words
print (all_words_frequency.most_common(10))

<FreqDist with 39768 samples and 1583820 outcomes>

[(',', 77717), ('the', 76529), ('.', 65876), ('a', 38106), ('and', 35576), ('of', 34123), ('to', 31937), ("'", 30585), ('is', 25195), ('in', 21822)]


In [9]:
stopwords_english = stopwords.words('english')
# create a new list of words by removing stopwords from all_words
all_words_without_stopwords = [word for word in all_words if word not in stopwords_english]
 
# print the first 10 words
print(all_words_without_stopwords[:10])

['plot', ':', 'two', 'teen', 'couples', 'go', 'church', 'party', ',', 'drink']


In [10]:
import string
# create a new list of words by removing punctuation from all_words
all_words_without_punctuation = [word for word in all_words if word not in string.punctuation]
 
# print the first 10 words
print (all_words_without_punctuation[:10])

['plot', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', 'drink']


In [11]:
all_words_clean = []
for word in all_words:
    if word not in stopwords_english and word not in string.punctuation:
        all_words_clean.append(word)
 
print(all_words_clean[:10])

['plot', 'two', 'teen', 'couples', 'go', 'church', 'party', 'drink', 'drive', 'get']


In [12]:
import spacy
nlp = spacy.load('en')
x = ' '.join(all_words_clean)
parse = nlp(x)

In [14]:
type(parse)

spacy.tokens.doc.Doc

In [15]:
all_words_frequency = FreqDist(all_words_clean)
 
print (all_words_frequency)

<FreqDist with 39586 samples and 710578 outcomes>


In [16]:
most_common_words = all_words_frequency.most_common(2000)
print (most_common_words[:10])
print('')
print (most_common_words[1990:])
print('')
word_features = [item[0] for item in most_common_words]
print (word_features[:10])

[('film', 9517), ('one', 5852), ('movie', 5771), ('like', 3690), ('even', 2565), ('good', 2411), ('time', 2411), ('story', 2169), ('would', 2109), ('much', 2049)]

[('exact', 64), ('causes', 64), ('niro', 64), ('international', 64), ('anna', 64), ('faced', 64), ('rain', 64), ('moved', 64), ('regular', 64), ('aware', 64)]

['film', 'one', 'movie', 'like', 'even', 'good', 'time', 'story', 'would', 'much']


In [18]:
documents[0]

(['october', 'sky', 'is', 'a', 'rare', 'oddity', 'in', ...], 'pos')

In [19]:
sentences = pd.DataFrame(documents)

In [20]:
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [21]:
from collections import Counter
# Set up the bags.
allwords = bag_of_words(parse)

In [23]:
# Combine bags to create a set of unique words.
common_words = set(allwords)

In [24]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

AttributeError: 'str' object has no attribute 'is_punct'